In [3]:
import cv2
import numpy as np
from ultralytics import YOLO
from sklearn.cluster import KMeans
from skimage import transform

In [4]:
#Intersección enter dos líneas
def line_intersection(line1, line2):
    x1, y1, x2, y2 = line1
    x3, y3, x4, y4 = line2

    # Calcular  pendientes e intersecciones
    denom = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
    if denom == 0:  # Líneas paralelas o coincidentes
        return None

    intersect_x = ((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3 * y4 - y3 * x4)) / denom
    intersect_y = ((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4)) / denom

    return (intersect_x, intersect_y)

#Comprueba que el punto esté dentro de la imagen
def is_within_image(point, image_width, image_height):
    x, y = point
    return 0 <= x < image_width and 0 <= y < image_height

def denormalizeCorners(corners, xmin, ymin):
    denormalized_corners = []
    for point in corners:
        x = int(point[0] + xmin)
        y = int(point[1] + ymin)
        denormalized_corners.append((x, y))
    return denormalized_corners

def groupCorners(denormalized_corners):
    corner_groups = []
    for i in range(len(denormalized_corners)):
        if(len(corner_groups) == 0):
            corner_groups.append([denormalized_corners[i]])
            continue
        # Revisa todas las nubes de puntos
        for j in range(len(corner_groups)):
            is_close = False
            for k in range(len(corner_groups[j])):
                if np.linalg.norm(np.array(denormalized_corners[i]) - np.array(corner_groups[j][k])) < 50:
                    corner_groups[j].append(denormalized_corners[i])
                    is_close = True
                    break
            if is_close:
                break
        # Si no se encuentra genera una nueva nube
        if not is_close:
            corner_groups.append([denormalized_corners[i]])

    return corner_groups

def filterCorners(corner_groups, frame_width, frame_height):
    ordered_points = []
    xmin = (frame_width,0)
    ymin = (0,frame_height)
    xmax = (0,0)
    ymax = (0,0)

    print("TEST")
    print(corner_groups)
    for i in range(len(corner_groups)):
        for j in range(len(corner_groups[i])):
            print(corner_groups[i][j])
            if(corner_groups[i][j][0] < xmin[0]):
                xmin = corner_groups[i][j]
            if(corner_groups[i][j][1] < ymin[1]):
                ymin = corner_groups[i][j]
            if(corner_groups[i][j][0] > xmax[0]):
                xmax = corner_groups[i][j]
            if(corner_groups[i][j][1] > ymax[1]):
                ymax = corner_groups[i][j]
    

    # Detectar en qué lado de la cancha está el aro
    # Si la intersección cuya x es mayor está a la derecha, entonces el aro está a la derecha
    is_right_side = xmax[0] > frame_width // 2

    # Punto A
    if is_right_side:
        # Punto A (xmin, y_A)
        print("Punto A: ", xmin)
        ordered_points.append(xmin)
    else:
        # Punto A (xmax, y_A)
        print("Punto A: ", xmax)
        ordered_points.append(xmax)

    # Estos puntos son independientes del lado de la cancha
    # Punto B (x_B, ymax)
    print("Punto B: ", ymax)
    ordered_points.append(ymax)

    # Punto C (x_C, ymin)
    print("Punto C: ", ymin)
    ordered_points.append(ymin)

    # Punto D
    if is_right_side:
        # Punto D (xmax, y_D)
        print("Punto D: ", xmax)
        ordered_points.append(xmax)

    else:
        # Punto D (xmin, y_D)
        print("Punto D: ", xmin)
        ordered_points.append(xmin)

    return ordered_points, is_right_side


def getZoneCorners(frame, model):

    # Realizar detección en el frame
    results = model.predict(frame, task='segment')
    # Dimensiones del frame original
    frame_height, frame_width = frame.shape[:2]

    # Se preestablecen las coordenadas del segmento a investigar
    xmin, ymin, xmax, ymax = 0,0,0,0
    cropped_zone = 0
    # Iterar sobre los resultados
    for result in results:
        masks = result.masks.data.cpu().numpy()  # Máscaras (NumPy array)
        classes = result.boxes.cls.cpu().numpy()  # Clases detectadas
        boxes = result.boxes.xyxy.cpu().numpy()  # Bounding boxes
        image = frame.copy()  # Crear copia del frame original para visualización

        # Aplicar y mostrar máscaras
        for i, cls in enumerate(classes):
            if model.names[cls] == 'three_second_area':
                # Redimensionar la máscara al tamaño del frame original
                zone_mask = (masks[i] * 255).astype(np.uint8)  # Escalar la máscara (0-255)
                resized_zone_mask = cv2.resize(zone_mask, (frame_width, frame_height), interpolation=cv2.INTER_NEAREST)

                # Invertir la máscara
                zone_mask_inv = cv2.bitwise_not(resized_zone_mask)

                # Aplicar la máscara al frame original
                combined = cv2.bitwise_and(image, image, mask=zone_mask_inv)
                
                xmin, ymin, xmax, ymax = boxes[i].astype(int)
                cropped_zone = combined[ymin:ymax, xmin:xmax]


    # Detección de líneas en la zona designada
    image = cropped_zone.copy()
    gaussian_image = cv2.GaussianBlur(image, (17,17), 0)

    # Forma utilizada
    mask = cv2.inRange(gaussian_image, 0,1)
    gaussian_masked_frame = cv2.GaussianBlur(mask, (17,17), 0)
    canny_masked_frame = cv2.Canny(gaussian_masked_frame, 30, 150)
    bilateral_canny_masked_frame = cv2.bilateralFilter(canny_masked_frame, 6, 75, 75)

    # Detecta líneas con la transformada de Hough
    min_line_length = canny_masked_frame.shape[0]*0.5  # Longitud mínima de la linea
    lineas_campo = cv2.HoughLinesP(bilateral_canny_masked_frame, rho=1, theta=np.pi / 180, threshold=15, minLineLength=min_line_length, maxLineGap=105)

    # Filtrar las nlineas líneas más próximas a la parte inferior de la imagen
    nlineas = 10
    if lineas_campo is not None:
        lineas_campo = sorted(lineas_campo, key=lambda l: min(l[0][1], l[0][3]), reverse=True)
        lineas_campo = lineas_campo[:10]

    # Inicializa grupos de líneas y umbral de orientación para separar
    group_1 = []  
    group_2 = []  
    angle_threshold = 20

    # Agrupa líneas por orientación
    if lineas_campo is not None:
        for line in lineas_campo:
            print(line)
            x1, y1, x2, y2 = line[0]
            
            # Evita división por cero en líneas verticales
            if x2 - x1 == 0:
                angle = 90  
            else:
                # Ángulo en grados
                angle = np.degrees(np.arctan((y2 - y1) / (x2 - x1)))
                angle = abs(angle) 
            
            # Asignar grupo
            if abs(angle) < angle_threshold:  # Cercano a horizontal
                group_1.append(line)
            else:  # Resto
                group_2.append(line)

    #INTERSECCIONES
    # Obtener las intersecciones entre líneas de grupo diferente
    intersections = []

    for i in range(len(group_1)):
        for j in range(len(group_2)):
            line1 = group_1[i][0]
            line2 = group_2[j][0]

            intersection = line_intersection(line1, line2)
            # Chequea que esté en la imagen
            if intersection:
                intersections.append(intersection)

    return intersections, xmin, ymin

def calculateHomography(zone_points, is_right_side):
    # Puntos del diagrama para la homografía: [A, B, C, D]
    diagram_points_right_side = [(1509, 436), (1509, 727), (1854, 436), (1854, 727)]
    diagram_points_left_side = [(490, 436), (490, 727), (145, 436), (145, 727)]

    if is_right_side:
        diagram_points = diagram_points_right_side
    else:
        diagram_points = diagram_points_left_side
    # Homografía
    return transform.estimate_transform('projective', np.array(zone_points), np.array(diagram_points))

def drawDetections(homography_matrix, diagram, team_A, team_B, referees):
    for point in team_A:
        diagram_point = homography_matrix(point)
        cv2.circle(diagram, (int(diagram_point[0][0]), int(diagram_point[0][1])), 15, getTeamBGRColor("A"), -1)

    for point in team_B:
        diagram_point = homography_matrix(point)
        cv2.circle(diagram, (int(diagram_point[0][0]), int(diagram_point[0][1])), 15, getTeamBGRColor("B"), -1)

    for point in referees:
        diagram_point = homography_matrix(point)
        cv2.circle(diagram, (int(diagram_point[0][0]), int(diagram_point[0][1])), 5, (255, 255, 255), -1)

In [7]:
# Función para obtener el ROI del área central de la bbox
def get_central_roi(frame, x1, y1, x2, y2):
    # Coordenadas de la bbox
    width = x2 - x1
    height = y2 - y1

    # Definir subárea como proporción de la bbox
    sub_x1 = int(x1 + width * 0.3)  # Dejar un 30% de margen a los lados
    sub_x2 = int(x2 - width * 0.3)  # Dejar un 30% de margen a los lados
    sub_y1 = int(y1 + height * 0.3)  # Subárea empieza al 30% de la altura
    sub_y2 = int(y1 + height * 0.6)  # Subárea llega al 60% de la altura

    # Recortar y devolver el ROI
    roi = frame[sub_y1:sub_y2, sub_x1:sub_x2]
    return roi

# Función para obtener los colores de los equipos
def classifyTeam(player_roi):
    
    # Convertir a HSV y preparar los datos para KMeans (matriz de (n_pixels, 3))
    roi_hsv = cv2.cvtColor(player_roi, cv2.COLOR_BGR2HSV)
    pixels = roi_hsv.reshape(-1, 3)

    # Aplicar KMeans para obtener los colores dominantes
    k = 2
    kmeans = KMeans(n_clusters=k, random_state=0).fit(pixels)
    dominant_colors = kmeans.cluster_centers_
    labels, counts = np.unique(kmeans.labels_, return_counts=True)

    # Obtener el color más dominante
    dominant_cluster = labels[np.argmax(counts)]
    dominant_color = dominant_colors[dominant_cluster]

    # Si no se han clasificado los colores de los equipos, asignar el color a uno de ellos
    if np.array_equal(team_colors['A'], np.array([0,0,0])):
        team_colors['A'] = dominant_color
        return 'A'
    
    # Si ya se ha clasificado el color de un equipo, asignar el color al otro
    elif np.array_equal(team_colors['B'], np.array([0,0,0])):
        # Si el color es muy similar al del equipo A, evita asignarlo a B
        if np.linalg.norm(dominant_color - team_colors['A']) < 100:
            return 'A'
        team_colors['B'] = dominant_color
        return 'B'
    
    # Si ya se han clasificado ambos colores, asignar el color al equipo más cercano
    else:
        if np.linalg.norm(dominant_color - team_colors['A']) < np.linalg.norm(dominant_color - team_colors['B']):
            return 'A'
        else:
            return 'B'

# Función para obtener el color BGR de un equipo a partir de su nombre
def getTeamBGRColor(team):
    if team not in team_colors:
        return [0, 0, 0]
    bgr_color = team_colors[team].astype(np.uint8)
    bgr_color = cv2.cvtColor(bgr_color[np.newaxis, np.newaxis, :], cv2.COLOR_HSV2BGR)[0, 0]
    return bgr_color.tolist()

def drawBBox(frame, x1, y1, x2, y2, team, label):
    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), getTeamBGRColor(team), 2)
    cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 200, 100), 2)

def drawPosition(frame, position, position_label):
    cv2.circle(frame, (int(position[0]), int(position[1])), 5, (0, 0, 255), -1)
    #cv2.ellipse(frame, (int(position[0]), int(position[1])), (15, 5), 0, 0, 360, (0, 0, 255), -1)
    cv2.putText(frame, position_label, (int(position[0]) - 50, int(position[1]) + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

def polygonArea(points):
    # Ensure there are exactly 4 points
    if len(points) != 4:
        raise ValueError("There must be exactly 4 points")

    # Shoelace formula
    x1, y1 = points[0]
    x2, y2 = points[1]
    x3, y3 = points[2]
    x4, y4 = points[3]

    area = 0.5 * abs(x1*y2 + x2*y3 + x3*y4 + x4*y1 - y1*x2 - y2*x3 - y3*x4 - y4*x1)
    return area

In [8]:
# Cargar el modelo YOLO entrenado
model = YOLO("./yolo_models/bpdv1.pt")
segment_model = YOLO("./yolo_models/segment.pt")

video_path = "./assets/videos/okc-nyk.mp4"
# Cargar el video de entrada
cap = cv2.VideoCapture(video_path)

# Obtener detalles del video
fourcc = cv2.VideoWriter_fourcc(*"avc1")  # Codec para el video de salida
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Configurar el video de salida
output_path = "video_resultado.mp4"
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

diagrama = cv2.imread('./assets/court_diagrams/white_court.png')

# Inicializar colores de los equipos
team_colors = {'A': np.array([0,0,0]), 'B': np.array([0,0,0])}
team_A = []
team_B = []
referees = []

# Inicializar puntos de la zona
zone_points = []
zone_detected = False

# Procesar cada frame del video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    diagramatmp = diagrama.copy()

    # Cálculo puntos de homografía
    corners, xmin, ymin = getZoneCorners(frame, segment_model)
    denormalized_corners = denormalizeCorners(corners, xmin, ymin)
    corner_groups = groupCorners(denormalized_corners)

    if(len(corner_groups) == 4):
        filtered_corners, is_right_side = filterCorners(corner_groups, width, height)
        if(polygonArea(filtered_corners) >= 800):
            zone_points = filtered_corners
            zone_detected = True

    if zone_detected:
        homography_matrix = calculateHomography(zone_points, is_right_side)
    
    # Realizar detección en el frame
    results = model(frame)

    # Análisis de las detecciones
    for result in results[0].boxes.data.tolist():  # Obtener los resultados como lista
        x1, y1, x2, y2, conf, cls = result  # Coordenadas, confianza y clase
        cls = int(cls)

        # Filtrar solo por las clases deseadas
        if model.names[cls] in ['player', 'referee'] and conf > 0.35:

            # Calcular la posición como el punto medio del borde inferior de la bbox
            position = (int((x1 + x2) / 2), y2)
            position_label = f"x:{int(position[0])} y:{int(position[1])}"

            label = f"{model.names[cls]} {conf:.2f}"

            # Si la clase es 'player', clasificar el equipo y dibujar la bbox en el frame
            if model.names[cls] == 'player':

                player_roi = get_central_roi(frame, int(x1), int(y1), int(x2), int(y2))

                team = classifyTeam(player_roi)

                drawBBox(frame, x1, y1, x2, y2, team, label)

                if team == 'A':
                    team_A.append(position)
                else:
                    team_B.append(position)


            # Si la clase es 'referee', dibujar la bbox en el frame
            if model.names[cls] == 'referee':
                drawBBox(frame, x1, y1, x2, y2, 'referee', label)
                referees.append(position)

            # Dibujar la posición en el frame
            drawPosition(frame, position, position_label)
    
    if zone_detected:
        drawDetections(homography_matrix, diagramatmp, team_A, team_B, referees)

    # Mostrar el frame procesado en pantalla
    cv2.imshow('Resultados', frame)

    cv2.imshow('Diagrama', diagramatmp)

    # Escribir el frame procesado en el video de salida
    out.write(frame)

    team_A.clear()
    team_B.clear()
    referees.clear()

    # Esperar 1ms para salir si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


# Liberar recursos
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Video procesado guardado en {output_path}")


0: 384x640 1 backboard, 1 backboard_inner_square, 1 court, 1 hoop, 1 net, 1 three_second_area, 1 two_point_area, 72.4ms
Speed: 2.0ms preprocess, 72.4ms inference, 4.5ms postprocess per image at shape (1, 3, 384, 640)
[[132 143 258 128]]
[[210 133 332 124]]
[[137 143 518  97]]
[[234 132 575  91]]
[[  9  64 120 141]]
[[  9  63 121 141]]
[[213  29 344  15]]
[[220  27 346  14]]
[[416  13 576  81]]
[[  9  53 414  10]]
TEST
[[(582, 453), (583, 453), (576, 449), (577, 449), (582, 454), (583, 454), (583, 455), (584, 454)], [(1050, 398), (1084, 412), (1050, 398), (1052, 398)], [(450, 362), (451, 362), (447, 360), (449, 360), (453, 364), (454, 364)], [(863, 318), (862, 317), (867, 320)]]
(582, 453)
(583, 453)
(576, 449)
(577, 449)
(582, 454)
(583, 454)
(583, 455)
(584, 454)
(1050, 398)
(1084, 412)
(1050, 398)
(1052, 398)
(450, 362)
(451, 362)
(447, 360)
(449, 360)
(453, 364)
(454, 364)
(863, 318)
(862, 317)
(867, 320)
Punto A:  (447, 360)
Punto B:  (583, 455)
Punto C:  (862, 317)
Punto D:  (108